In [ ]:
import json
from data_collection import get_fonts_info

# Load the configuration file
with open('../../config/config.json') as config_file:
    config = json.load(config_file)

fonts_df = get_fonts_info(config['GOOGLE_FONTS_API_KEY'])

In [ ]:
from data_collection import select_fonts, download_fonts
import os

fonts_path = "../../data/raw/fonts/"

Download=True
if Download:
    fonts_to_download = select_fonts(fonts_df, subsets=['hebrew', 'arabic'])
    fonts_to_download = download_fonts(fonts_to_download, os.path.join(fonts_path, "GF"))
    if not os.path.exists("data.csv"):
        fonts_to_download.to_csv("data.csv", index=False)
    else:
        fonts_to_download.to_csv("data.csv", mode='a', index=False, header=False)


In [ ]:
import pandas as pd

def parse_list(string: str) -> list:
    """Convert a string representation of a list to a list."""
    return [s.strip("''") for s in string.strip('[]').split(', ')]

# Read the CSV file, using the parse_list function to parse the values in the "subsets" and "file_path" columns
df = pd.read_csv("data.csv", converters={"subsets": parse_list, "file_path": parse_list})

# Drop irrelevant columns
df = df.drop(["variants", "version", "lastModified", "files", "kind"], axis=1)

In [ ]:
from tqdm import tqdm
    
# Iterate through the file paths in the 'file_path' column
#TODO TEST FAILURE: df = df[df.family == "Amiri Quran"]
for _, row in tqdm(df.iterrows()):
    # Get the values in the "file_path" and "family" columns
    file_path = row["file_path"]
    family = row["family"]

    print(family)
    for ttf_file_path in file_path:
        # Get the file name and extension of the TTF file
        ttf_file_name, ttf_file_extension = os.path.splitext(ttf_file_path)
        # Set the file path of the UFO file
        family_folder = os.path.join(fonts_path, f"UFO/{family}")
        
        if not os.path.exists(family_folder):
            os.makedirs(family_folder)

        variant = ttf_file_name.split("-")[-1]

        ufo_file_path = os.path.join(family_folder, f"{family}-{variant}.ufo")

        # Convert the TTF file to a UFO file
        ttf_to_ufo(ttf_file_path, ufo_file_path)
